# 回归

## 读入文件

In [67]:
import netCDF4

import numpy as np

f = netCDF4.Dataset('D:\\sst.mnmean.nc')  # ftp://ftp.cdc.noaa.gov/Datasets/noaa.ersst.v5/sst.mnmean.nc
SST = f.variables['sst'][:, :, :].data

In [68]:
'''
从1854年1月到2019年2月，总计1982个月;
栅格中心点从88°N到-88°N（88°S），0°到358°E（2°W）;
栅格大小为2°×2°
'''
np.shape(SST) 

(1982, 89, 180)

In [70]:
# 134, 44 表示（134 * 2）°E→92°W, （88 - 44 * 2）°N→0°
loc = ((134, 44), \
       (94, 44), \
       (57, 37), \
       (39, 44), \
       (21, 36), \
       (9, 16), \
       (172, 59), \
       (158, 28), \
       (65, 64), \
       (0, 10))

for row in loc:
    print(SST[-1, row[1], row[0]])

27.236326
28.912207
26.859951
29.2531
27.29979
3.3593714
25.75996
20.125439
16.212143
6.4317055


In [71]:
SST.max()

42.32636

In [72]:
SST.min()

-9.96921e+36

In [73]:
SST[SST == SST.min()] = 999999
SST.min()

-1.8

In [74]:
A = SST < 999999  # 1表示海洋，0表示陆地
A = np.average(A, axis = 0)
np.sum(A)

10988.0

## 数据预处理

In [75]:
new_SST = np.zeros((len(SST), int(np.sum(A))))  # 合并SST数据从一个三维数组到一个二维数组

s = 0

for i in range(89):
    for j in range(180):
        if A[i, j] == 1:
            if (j, i) in loc:
                print(j, i, ':', s)
            new_SST[:, s] += SST[:, i, j]
            s += 1

0 10 : 1511
9 16 : 1925
158 28 : 3160
21 36 : 4012
57 37 : 4174
39 44 : 5164
94 44 : 5215
134 44 : 5255
172 59 : 7401
65 64 : 8129


In [76]:
new_SST = new_SST[-1202:-2]  # 选取1919年1月至2018年12月1200个月的数据

## 求月平均

In [77]:
month_mean = np.zeros((12, int(np.sum(A))))
month_s = np.arange(0, 1200, 12)
month_s

array([   0,   12,   24,   36,   48,   60,   72,   84,   96,  108,  120,
        132,  144,  156,  168,  180,  192,  204,  216,  228,  240,  252,
        264,  276,  288,  300,  312,  324,  336,  348,  360,  372,  384,
        396,  408,  420,  432,  444,  456,  468,  480,  492,  504,  516,
        528,  540,  552,  564,  576,  588,  600,  612,  624,  636,  648,
        660,  672,  684,  696,  708,  720,  732,  744,  756,  768,  780,
        792,  804,  816,  828,  840,  852,  864,  876,  888,  900,  912,
        924,  936,  948,  960,  972,  984,  996, 1008, 1020, 1032, 1044,
       1056, 1068, 1080, 1092, 1104, 1116, 1128, 1140, 1152, 1164, 1176,
       1188])

In [78]:
for i in range(12):
    month_mean[i] = np.copy(np.average(new_SST[month_s + i, :], axis = 0))
month_mean[:, 1925]  # 波罗的海某处海域100年月平均海温

array([ 3.82858563,  2.70515369,  2.5106136 ,  3.79849035,  7.37022211,
       12.43795303, 16.50817428, 17.54718121, 14.94804304, 11.39447208,
        8.28368125,  5.8330507 ])

注意，此处是对全部数据（训练集+验证集）取得月平均，但在实际情况下，只能对训练集取月平均，因为验证集数据不能在训练中使用

In [79]:
for i in range(12):
    new_SST[month_s + i, :] -= month_mean[i]  # 每月减掉月平均

## 整理为时间序列

In [80]:
p = 12  # p为时间序列的长度
X = np.zeros((len(new_SST) - p, 10988 * p))
np.shape(X)

(1188, 131856)

In [130]:
for i in range(p):
    for j in range(10988):
        X[:, i * 10988 + j] = np.copy(new_SST[i:1200-p+i, j])
y = np.copy(new_SST[p:, 1925])

## PCA

更多关于sklearn的教程访问https://scikit-learn.org/stable/index.html

In [86]:
from sklearn.decomposition import PCA

pca = PCA(n_components=100)  # 设置PCA想要降到的维数

X_mean = np.average(X, axis = 0)  # 对每个变量求平均值

np.shape(X_mean)

(131856,)

In [87]:
X -= X_mean  # 做PCA前需要先减掉均值
pca.fit(X)

PCA(copy=True, iterated_power='auto', n_components=100, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

注意，此处是对全部数据做PCA，但实际情况下只能对训练数据做PCA

In [90]:
pca.explained_variance_ratio_

array([0.17053385, 0.06945562, 0.03655253, 0.02804149, 0.01945243,
       0.01653311, 0.01533042, 0.01208452, 0.01197686, 0.01161306,
       0.0105101 , 0.00990932, 0.0088608 , 0.00844643, 0.00796712,
       0.00763522, 0.00732797, 0.00689304, 0.00678529, 0.00659345,
       0.00644957, 0.00617663, 0.00601633, 0.0057301 , 0.00568446,
       0.00552521, 0.00541347, 0.00529244, 0.00522196, 0.00516523,
       0.00498095, 0.00492239, 0.00480719, 0.00472453, 0.00459506,
       0.00445903, 0.00429184, 0.00419281, 0.00408098, 0.00396984,
       0.00385754, 0.00383372, 0.00376115, 0.00368804, 0.00351464,
       0.00337239, 0.00329037, 0.00325257, 0.003178  , 0.0031073 ,
       0.00304018, 0.00300429, 0.00295057, 0.00292768, 0.00286978,
       0.00281226, 0.0027259 , 0.0027017 , 0.00265362, 0.00260219,
       0.00257846, 0.00254121, 0.00250617, 0.00248451, 0.00242527,
       0.00240268, 0.00239685, 0.00238259, 0.00233003, 0.00228865,
       0.00226014, 0.00222308, 0.00220616, 0.00216058, 0.00214

In [92]:
new_X = pca.fit_transform(X)  # 将X降至100维的new_X

## 线性回归

In [137]:
from sklearn import linear_model

reg = linear_model.LinearRegression()

reg.fit(new_X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [139]:
y_fore = np.sum(new_X * reg.coef_, axis=1) + reg.intercept_  # 注意此时y是距平后的y

In [140]:
np.average(abs(y_fore - y))  # MAE

0.5818437105791892

In [141]:
np.average((y_fore - y) ** 2) ** 0.5  #RMSE

0.7356869704092893

In [142]:
y_true = np.copy(y)
for i in range(12):
    y_true[month_s[:-1] + i] += month_mean[i, 1925]
    y_fore[month_s[:-1] + i] += month_mean[i, 1925]

In [143]:
1 - np.sum((y_fore - y_true) ** 2) / np.sum((y_true - np.average(y_true)) ** 2)  # 还原y后做NSE

0.9812245843802776

## 交叉验证

In [146]:
import sklearn.model_selection as sk_model_selection

sk_model_selection.cross_val_score(reg, new_X, np.array([y]).T, scoring='neg_mean_squared_error', cv = 10)  # 10折RMSE

array([-1.22041248, -1.50091526, -1.1983714 , -0.80604046, -1.1666982 ,
       -0.84625854, -0.75093762, -1.00802402, -0.68830928, -0.7926849 ])

In [147]:
sk_model_selection.cross_val_score(reg, new_X, np.array([y]).T, scoring='neg_mean_absolute_error', cv = 10)  # 10折MAE

array([-0.89187518, -0.97277393, -0.82996782, -0.73620573, -0.86147554,
       -0.74021235, -0.70602609, -0.77289089, -0.66517877, -0.68821656])

## LASSO回归

In [153]:
reg = linear_model.LinearRegression()

for i in (0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 100, 1000):
    reg = linear_model.Lasso(alpha=i, max_iter=10000)
    print(i, abs(np.mean(sk_model_selection.cross_val_score(reg, new_X, np.array([y]).T, scoring='neg_mean_squared_error', cv = 10)))**0.5, abs(np.mean(sk_model_selection.cross_val_score(reg, new_X, np.array([y]).T, scoring='neg_mean_absolute_error', cv = 10))))

1e-05 0.9989273870321032 0.786478051450857
0.0001 0.9988854713808079 0.7864398796940597
0.001 0.9984706495858549 0.7860595169930045
0.01 0.9946307551431194 0.7825272186430725
0.1 0.9781352874496574 0.7681469267044095
1 1.0209880186191678 0.8043228140602429
100 1.1288610004990325 0.8883027853272513
1000 1.1288610004990325 0.8883027853272513


## 岭回归

In [156]:
reg = linear_model.LinearRegression()

for i in (0.01, 0.1, 1, 100, 1000, 10000, 100000, 1000000):
    reg = linear_model.Ridge(alpha=i, max_iter=10000)
    print(i, abs(np.mean(sk_model_selection.cross_val_score(reg, new_X, np.array([y]).T, scoring='neg_mean_squared_error', cv = 10)))**0.5, abs(np.mean(sk_model_selection.cross_val_score(reg, new_X, np.array([y]).T, scoring='neg_mean_absolute_error', cv = 10))))

0.01 0.9989320197118629 0.7864822683161475
0.1 0.9989318590364005 0.7864821104202532
1 0.9989302523481902 0.786480531481001
100 0.9987542514934634 0.7863070663809391
1000 0.9972189954193057 0.7847689683418514
10000 0.9867162636024968 0.7739578944436715
100000 0.98495080483064 0.7698110176439829
1000000 1.0318701069585452 0.8139866290794385


## 数据保存

In [157]:
reg = linear_model.Lasso(alpha=0.01, max_iter=10000)
reg.fit(new_X, y)
y_fore = np.sum(new_X * reg.coef_, axis=1) + reg.intercept_  # 注意此时y是距平后的y
for i in range(12):
    y_fore[month_s[:-1] + i] += month_mean[i, 1925]
1 - np.sum((y_fore - y_true) ** 2) / np.sum((y_true - np.average(y_true)) ** 2)  # 还原y后做NSE

0.9812222075876501

In [158]:
np.savetxt('y_fore.csv', y_fore, delimiter = ',')